In [1]:
from ASmartis import * 
from ipywidgets import HBox,VBox,Layout

class Wechselstromrechnung(ASmartis):
    
    def __init__(self,parameterList):
        self.xPositionMin = 50
        self.xPositionMax = 950
        
        self.yFrameOrigin = 400
        
        
        
        self.xPositionField1 = 200
        self.yPositionField1 = 0
        self.xPositionField2 = 450
        self.yPositionField2 = 0
        self.xPositionField3 = 700
        self.yPositionField3 = 0
        
        self.widthField  = 100
        self.heightField = 180
        
        self.widthConductor = 10
        self.lengthConductor = 100
        
        #self.stopSendInformation()
        
        super().__init__(parameterList)
        self.updateTime = 0.08
        
    def __del__(self):
        super().__del__()
        
    def createButtons(self):        
        self.buttonsObject = Buttons(self)
        self.buttonsObject.newReset("b_reset", 30,30)
        self.buttonsObject.newTogglePlayPause("b_play", 30,30)
        
        self.buttonsObject.newIntSlider("s_b-field1", -5,5,0,
            r"$\color{#00c000}{\vec{B_1}}$",newOrientation='vertical',newReadout=False)
        self.buttonsObject.newIntSlider("s_b-field2", -5,5,0,
            r"$\color{#00c000}{\vec{B_2}}$",newOrientation='vertical',newReadout=False)
        self.buttonsObject.newIntSlider("s_b-field3", -5,5,0,
            r"$\color{#00c000}{\vec{B_3}}$",newOrientation='vertical',newReadout=False)
        self.buttonsObject.newIntSlider("s_length", 10,100,10,r"Länge $l$",newReadout=False)
        self.buttonsObject.newIntSlider("s_width", 10,40,10,r"Breite $b$",newReadout=False)
        self.buttonsObject.newIntSlider("s_velocity", 1,3,1,r"$v$",newReadout=False)
        self.buttonsObject.newIntSlider("s_position", self.xPositionMin,self.xPositionMax,0,r"$x$",newReadout=False)        
        
        self.buttonsStatesDict = self.buttonsObject.getStates()
        self.oldButtonsStatesDict = dict(self.buttonsStatesDict)
        
            
    def createChart(self):        
        self.layers = ["background","field1under","field2under","field3under","conductorLine","conductor", "conductorCurrent","field1up","field2up","field3up", "voltagePredicted", "voltageActual"]        
        self.chartObject = Chart(self.layers, 1000, 600)
        
        for currentLayer in self.layers:
            self.chartObject.changeLayerLook(currentLayer,"lineColor","#00d000")
            self.chartObject.changeLayerLook(currentLayer,"fillingColor","#00d000")
 

        
        self.chartObject.changeLayerLook("voltagePredicted","lineColor","#3333cc")
        self.chartObject.changeLayerLook("voltagePredicted","lineDash", [10,3])
        
        self.chartObject.changeLayerLook("voltageActual","lineColor","#3333cc")
        self.chartObject.changeLayerLook("voltageActual","fillingColor","#3333cc")
        self.chartObject.changeLayerLook("voltageActual","lineWidth",2)
        
        self.chartObject.changeLayerLook("conductorLine","lineColor","#3333cc")
        self.chartObject.changeLayerLook("conductorLine","lineDash", [10,3])
        
        self.drawBackground()
        self.reset()
        
     
        
    def visualize(self):          
        chart = self.chartObject.getChart()
        buttonsDict = self.buttonsObject.getButtons()
        emptyBox = VBox([widgets.Label(value="")])
        
        buttonsBox = HBox([buttonsDict["b_play"],buttonsDict["b_reset"],buttonsDict["s_position"]])
        sliderBox = VBox([buttonsDict["s_length"],buttonsDict["s_width"],
            buttonsDict["s_velocity"]])
        
        fieldSliderBox = HBox([emptyBox,buttonsDict["s_b-field1"],buttonsDict["s_b-field2"],buttonsDict["s_b-field3"],emptyBox],
            layout=Layout(display='flex', justify_content = 'space-around',layout=Layout))
        
        display(VBox([fieldSliderBox, chart,buttonsBox, sliderBox], layout=Layout(width='60%', height='min_height')))
        
    def update(self):                     
        if self.buttonsStatesDict["b_reset"]:
            self.reset()            
        else:
            changeInVoltage = False
            changeInVoltageActual = False
            changeAtConductor = False
            
            if self.buttonsStatesDict["b_play"]:
                self.play()                
            
            if self.buttonsStatesDict["s_b-field1"] != self.oldButtonsStatesDict["s_b-field1"]:
                self.drawField(1,"s_b-field1",self.xPositionField1,self.yPositionField1)
                changeInVoltage = True
                changeInVoltageActual = True
                
            if self.buttonsStatesDict["s_b-field2"] != self.oldButtonsStatesDict["s_b-field2"]:
                self.drawField(2,"s_b-field2",self.xPositionField2,self.yPositionField2)
                changeInVoltage = True
                changeInVoltageActual = True
                
            if self.buttonsStatesDict["s_b-field3"] != self.oldButtonsStatesDict["s_b-field3"]:
                self.drawField(3,"s_b-field3",self.xPositionField3,self.yPositionField3)
                changeInVoltage = True
                changeInVoltageActual = True
                
            if (self.buttonsStatesDict["s_width"] != self.oldButtonsStatesDict["s_width"] or
                self.buttonsStatesDict["s_length"] != self.oldButtonsStatesDict["s_length"]):                
                changeInVoltage = True
                changeInVoltageActual = True
                changeAtConductor = True
                 
            if self.buttonsStatesDict["s_velocity"] != self.oldButtonsStatesDict["s_velocity"]:
                changeInVoltage = True
                changeInVoltageActual = True
                
            if self.buttonsStatesDict["s_position"] != self.oldButtonsStatesDict["s_position"]:
                changeInVoltageActual = True
                changeAtConductor = True
                
            if changeInVoltage:
                self.calculateVoltage()
                self.drawVoltage()
            if changeInVoltageActual:
                self.drawVoltageActual()
            if changeAtConductor:
                self.drawConductor()
            
    def reset(self):
        
        self.buttonsObject.changeState("b_play",False)
        self.buttonsObject.changeState("s_b-field1",0)
        self.buttonsObject.changeState("s_b-field2",-5)
        self.buttonsObject.changeState("s_b-field3",0)
        self.buttonsObject.changeState("s_length",50)
        self.buttonsObject.changeState("s_width",20)
        self.buttonsObject.changeState("s_velocity",2)
        self.buttonsObject.changeState("s_position",self.xPositionMin)
        
        
        self.drawField(1,"s_b-field1",self.xPositionField1,self.yPositionField1)
        self.drawField(2,"s_b-field2",self.xPositionField2,self.yPositionField2)
        self.drawField(3,"s_b-field3",self.xPositionField3,self.yPositionField3)
        
        self.calculateVoltage()
        self.drawVoltage()
        self.drawVoltageActual()
        self.drawConductor()
        
        
        self.buttonsObject.changeState("b_reset",False)
        
        
    
    def drawBackground(self):
        currentLayer = "background"
        self.chartObject.clearLayer(currentLayer)
        
        #Koordinatensytem
        self.chartObject.changeLayerLook(currentLayer,"lineColor","#000000")
        self.chartObject.changeLayerLook(currentLayer,"fillingColor","#000000")
        self.chartObject.drawArrow(currentLayer,self.xPositionMin, self.yFrameOrigin,self.xPositionMax, self.yFrameOrigin,10,6)
        self.chartObject.drawArrow(currentLayer,self.xPositionMin, 600,self.xPositionMin, 200,10,6)
        
        #Bezeichnungen fürs Koordinatensytem
        self.chartObject.drawText(currentLayer,self.xPositionMax+10, 400, '20px serif', "x")        
        
        self.chartObject.changeLayerLook(currentLayer,"fillingColor","#3333cc")
        self.chartObject.drawText(currentLayer,self.xPositionMin-30, 200, '20px serif', "u")
        self.chartObject.drawText(currentLayer,self.xPositionMin-20, 205, '15px serif', "ind")
        
        #Abgrenzungen für das B-Feld
        self.chartObject.changeLayerLook(currentLayer,"lineColor","#00d000")
        self.chartObject.changeLayerLook(currentLayer,"fillingColor","#ffffff")
        self.chartObject.changeLayerLook(currentLayer,"lineDash",[10,3])
        self.chartObject.drawRectangle(currentLayer,self.xPositionField1, self.yPositionField1,
            self.widthField, self.heightField)
        self.chartObject.drawRectangle(currentLayer,self.xPositionField2, self.yPositionField2,
            self.widthField, self.heightField)
        self.chartObject.drawRectangle(currentLayer,self.xPositionField3, self.yPositionField3,
            self.widthField, self.heightField)
        
        #Bewegungsbahn für die Leiterschleife
        self.chartObject.changeLayerLook(currentLayer,"lineColor","#c0c0c0")
        self.chartObject.drawLine(currentLayer,0, self.heightField/2,1000, self.heightField/2)
        
         
        self.chartObject.update(currentLayer)
    
    def play(self):
        if self.buttonsStatesDict["s_position"]<self.xPositionMax:
        
            self.buttonsObject.changeState("s_position",
                    self.buttonsStatesDict["s_position"]+2*self.buttonsStatesDict["s_velocity"])
        else:
            self.buttonsObject.changeState("s_position",self.xPositionMin)
            self.buttonsObject.changeState("b_play",False)
            
        
    
    def drawField(self,fieldNumber,sliderName,x,y):
        
        upperLayer = "field"+str(fieldNumber)+"up"
        underLayer = "field"+str(fieldNumber)+"under"
        
        width = self.widthField
        height = self.heightField
        
        number = abs(self.buttonsStatesDict[sliderName])
        
        self.chartObject.clearLayer(upperLayer)
        self.chartObject.drawParallelLines(upperLayer,x, y,
            width, height/2, number,"vertical")
        
        
        if self.buttonsStatesDict[sliderName]>0:
            y = y+self.heightField
            height = -self.heightField           
        
        self.chartObject.clearLayer(underLayer)
        self.chartObject.drawParallelArrows(underLayer,x, y,
            width, height, number,"vertical",10,6)
        
        self.chartObject.update(underLayer,upperLayer)
        
    def drawConductor(self):
        
        currentLayer = 'conductor'
        self.chartObject.clearLayer(currentLayer)
        
        #Zeichnen senkrechte Linie
        self.chartObject.changeLayerLook(currentLayer,"lineColor","#3333cc")
        self.chartObject.changeLayerLook(currentLayer,"fillingColor","#3333cc")
        self.chartObject.changeLayerLook(currentLayer,"lineDash", [10,3])
        self.chartObject.changeLayerLook(currentLayer,"lineWidth",1.5)
        
        edges = self.edgesTuple
        x = self.buttonsStatesDict["s_position"] 
        y = edges[0][1]    
        for point in range(len(edges)-1):
            if (x>edges[point][0] and x<edges[point+1][0]):
                y = edges[point][1]
                break
        self.chartObject.drawLine(currentLayer,x,y,x,self.heightField/2)
        self.chartObject.drawCircle(currentLayer,x,y,5)
        
        #Zeichnen Leiteschleife        
        width = self.buttonsStatesDict["s_width"]
        length = self.buttonsStatesDict["s_length"]
        xPosition = self.buttonsStatesDict["s_position"]
        
        
        x1 = xPosition-length/2+width/2
        y1 = self.heightField/2-width/2
        x2 = xPosition+length/2+width/2
        y2 = self.heightField/2-width/2
        x3 = xPosition+length/2-width/2
        y3 = self.heightField/2+width/2
        x4 = xPosition-length/2-width/2
        y4 = self.heightField/2+width/2 
        
        self.chartObject.changeLayerLook(currentLayer,"lineColor","#000000")
        self.chartObject.changeLayerLook(currentLayer,"fillingColor","#c0c0c0")
        self.chartObject.changeLayerLook(currentLayer,"lineDash", [0,0])       
        self.chartObject.drawPolygon(currentLayer,(x1,y1), (x2,y2),(x3,y3), (x4,y4)) 

        #Zeichnen rote Kanten
        leftSide = xPosition-length/2
        rightSide = xPosition+length/2
                                         
        self.chartObject.changeLayerLook(currentLayer,"lineColor","#FF0000")
        self.chartObject.changeLayerLook(currentLayer,"lineWidth",2)
        
        if (leftSide > self.xPositionField1 and leftSide < self.xPositionField1+self.widthField or
            leftSide > self.xPositionField2 and leftSide < self.xPositionField2+self.widthField or
            leftSide > self.xPositionField3 and leftSide < self.xPositionField3+self.widthField):
            self.chartObject.drawLine(currentLayer,x1,y1,x4,y4)
            
        if (rightSide > self.xPositionField1 and rightSide < self.xPositionField1+self.widthField or
            rightSide > self.xPositionField2 and rightSide < self.xPositionField2+self.widthField or
            rightSide > self.xPositionField3 and rightSide < self.xPositionField3+self.widthField):
            self.chartObject.drawLine(currentLayer,x2,y2,x3,y3)       

            
        self.chartObject.update(currentLayer)
        
    def calculateVoltage(self):
        
        l = self.buttonsStatesDict["s_length"]
        v = self.buttonsStatesDict["s_velocity"]
        b = self.buttonsStatesDict["s_width"]/5
        B1 = self.buttonsStatesDict["s_b-field1"]
        B2 = self.buttonsStatesDict["s_b-field2"]
        B3 = self.buttonsStatesDict["s_b-field3"]
        
        u_ind = 1000*[self.yFrameOrigin]
        B_list = []
        
        
        for x in range(self.xPositionMin,self.xPositionMax):
            u = self.yFrameOrigin
            if ((x+l/2) > self.xPositionField1 and 
                (x+l/2) < (self.xPositionField1+self.widthField)):
                u = u+v*b*B1 
            if ((x-l/2) > self.xPositionField1 and 
                (x-l/2) < (self.xPositionField1+self.widthField)):
                u = u-v*b*B1
            if ((x+l/2) > self.xPositionField2 and 
                (x+l/2) < (self.xPositionField2+self.widthField)):
                u = u+v*b*B2                
            if ((x-l/2) > self.xPositionField2 and 
                (x-l/2) < (self.xPositionField2+self.widthField)):
                u = u-v*b*B2                
            if ((x+l/2) > self.xPositionField3 and 
                (x+l/2) < (self.xPositionField3+self.widthField)):
                u = u+v*b*B3                
            if ((x-l/2) > self.xPositionField3 and 
                (x-l/2) < (self.xPositionField3+self.widthField)):
                u = u-v*b*B3
            u_ind[x]=u
        
        self.edgesTuple = [(self.xPositionMin,self.yFrameOrigin)]
        
        for x in range(self.xPositionMin,self.xPositionMax):
            if (u_ind[x] != u_ind[x+1]):
                self.edgesTuple.append((x,u_ind[x]))
                self.edgesTuple.append((x+1,u_ind[x+1]))
        self.edgesTuple.append((self.xPositionMax,self.yFrameOrigin))
        
  
            
   
    def drawVoltage(self):
        edges = self.edgesTuple
        
        x_start = self.xPositionMin
        y = self.yFrameOrigin
        
        currentLayer = "voltagePredicted"
        self.chartObject.clearLayer(currentLayer)
        
        for point in range(len(edges)-1):
            self.chartObject.drawLine(currentLayer,edges[point][0],edges[point][1], 
                                      edges[point+1][0], edges[point+1][1])  
            
        self.chartObject.update(currentLayer)
        
    def drawVoltageActual(self):
        
        edges = self.edgesTuple
        
        voltageLayer = "voltageActual"    
        self.chartObject.clearLayer(voltageLayer)
        
        for point in range(len(edges)-1):
            if self.buttonsStatesDict["s_position"]>=edges[point+1][0]:
                self.chartObject.drawLine(voltageLayer,edges[point][0],edges[point][1],
                                      edges[point+1][0], edges[point+1][1])
            else:
                self.chartObject.drawLine(voltageLayer,edges[point][0],edges[point][1],
                                      self.buttonsStatesDict["s_position"], edges[point][1])
                break
                    
        self.chartObject.update(voltageLayer)
    
        
        
   

In [2]:
if __name__ == '__main__':
    ex = Leiterschleife(None)
    ex.visualize()
    ex.start()
    
    